In [1]:
import xgi
H = xgi.read_hif('/Users/ambu249/Library/CloudStorage/OneDrive-PNNL/Documents/nick_collab/a-blue-start/data/deidentified_starterpack_hif.json')

d_h = H.degree()

In [5]:
import csv
from collections import defaultdict

def count_node_degrees(file_path):
    node_degrees = defaultdict(int)

    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            if i % 10000000 == 0:
                print(f"Processed {i} rows")
            if len(row) != 2:
                continue
            node1, node2 = row
            node_degrees[node1] += 1
            node_degrees[node2] += 1

    return dict(node_degrees)

d_g = count_node_degrees('/Users/ambu249/Library/CloudStorage/OneDrive-PNNL/Documents/nick_collab/a-blue-start/data/deidentified_follows_edgelist.csv') 

Processed 0 rows
Processed 10000000 rows
Processed 20000000 rows
Processed 30000000 rows
Processed 40000000 rows
Processed 50000000 rows
Processed 60000000 rows
Processed 70000000 rows
Processed 80000000 rows
Processed 90000000 rows
Processed 100000000 rows
Processed 110000000 rows
Processed 120000000 rows
Processed 130000000 rows
Processed 140000000 rows
Processed 150000000 rows
Processed 160000000 rows
Processed 170000000 rows
Processed 180000000 rows
Processed 190000000 rows
Processed 200000000 rows
Processed 210000000 rows
Processed 220000000 rows
Processed 230000000 rows
Processed 240000000 rows
Processed 250000000 rows
Processed 260000000 rows
Processed 270000000 rows
Processed 280000000 rows
Processed 290000000 rows
Processed 300000000 rows
Processed 310000000 rows
Processed 320000000 rows
Processed 330000000 rows
Processed 340000000 rows
Processed 350000000 rows
Processed 360000000 rows
Processed 370000000 rows
Processed 380000000 rows
Processed 390000000 rows
Processed 4000000

In [ ]:
d_g = {int(k): v for k, v in d_g.items()}

In [10]:
import pandas as pd

def write_dict_to_csv_gz(d, filename, key_label="key", value_label="value"):
    df = pd.DataFrame(list(d.items()), columns=[key_label, value_label])
    df.to_csv(filename, index=False, compression="gzip")
write_dict_to_csv_gz(d_h, 'data/starterpack_degree.csv.gz', key_label='node', value_label='degree')
write_dict_to_csv_gz(d_g, 'data/follows_degree.csv.gz', key_label='node', value_label='degree')

In [ ]:
import numpy as np
from fig_settings import set_fonts
set_fonts()
def compare_top_k(d_g, d_h, k):
    top_g = sorted(d_g.items(), key=lambda x: x[1], reverse=True)[:k]
    top_h = sorted(d_h.items(), key=lambda x: x[1], reverse=True)[:k]
    
    top_g_nodes = {node for node, _ in top_g}
    top_h_nodes = {node for node, _ in top_h}
    
    common_nodes = top_g_nodes.intersection(top_h_nodes)
    
    common_nodes_list = sorted(common_nodes)
    
    cent1 = np.array([d_g[node] for node in common_nodes_list])
    cent2 = np.array([d_h[node] for node in common_nodes_list])
    
    return common_nodes_list, cent1, cent2

common_nodes_list, cent1, cent2 = compare_top_k(d_g, d_h, k=1000000)

In [40]:
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns

colors = ["#2d5d83", "#4f9bd9", "#077187", "#3dfaff", "#8b888e"]

def _log_bin_stats(x, y, nbins=30, reducer=np.mean):
    edges = np.logspace(np.log10(x.min()), np.log10(x.max()), nbins + 1)
    centres = np.sqrt(edges[:-1] * edges[1:])          # geometric mid‑points
    y_binned = np.full(nbins, np.nan)

    for i, (lo, hi) in enumerate(zip(edges[:-1], edges[1:])):
        mask = (x >= lo) & (x < hi)
        if mask.any():
            y_binned[i] = reducer(y[mask])

    ok = ~np.isnan(y_binned)
    return centres[ok], y_binned[ok]

def rank_corr(
    cent1,
    cent2,
    index_start,
    dataset,
    cent1_label="cent1",
    cent2_label="cent2",
    nbins=30,
):
    ran = np.logspace(np.log10(index_start - 1),
                      np.log10(len(cent1) - 1),
                      500,
                      dtype=int)

    # tau(cent1 | cent2‑ordering)
    sp = np.argsort(cent1)
    c1, c2 = cent1[sp], cent2[sp]
    tau_12 = np.array([ss.kendalltau(c1[-s:], c2[-s:], variant="b")[0] for s in ran])

    # tau(cent2 | cent1‑ordering)
    sp = np.argsort(cent2)
    c1, c2 = cent1[sp], cent2[sp]
    tau_21 = np.array([ss.kendalltau(c2[-s:], c1[-s:], variant="b")[0] for s in ran])

    xs = ran + 1

    x1, y1 = _log_bin_stats(xs, tau_12, nbins=nbins)
    x2, y2 = _log_bin_stats(xs, tau_21, nbins=nbins)

    _, ax = plt.subplots(figsize=(8, 5))
    ax.set_xscale("log")

    ax.plot(x1, y1, marker="o", linestyle="None",
            label=fr"{cent1_label} - {cent2_label}", color=colors[0])

    ax.plot(x2, y2, marker="^", linestyle="None",
            label=fr"{cent2_label} - {cent1_label}", color=colors[1])

    ax.set_xlabel("Number of Top-Ranked Elements, $k$")
    ax.set_ylabel("Kendall's $\\tau_B$ Rank Corr. Coefficient")
    ax.set_title(dataset)
    ax.tick_params(axis="both")
    ax.legend()
    sns.despine()

    plt.savefig("figures/kendalltaurankcorr.pdf", bbox_inches="tight")
    plt.close()


In [30]:
rank_corr(cent1, cent2, index_start=2, dataset="",
          cent1_label=r"$d_G$", cent2_label=r"$d_H$", nbins=50)